In [1]:
import os

from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_extractor import TableExtractor
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper
from series_extraction.excel_validator import ExcelValidator

from ast_transformation.formula_generator import FormulaGenerator

from ast_building.formula_parser import FormulaParser

from excel_builder import ExcelBuilder

In [2]:
data_directory = "/Users/chrislittle/GitHub/speedsheet/excel-2-python/data"

project_name = 'test_excel_2'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")
excel_reduced_clean_filepath = os.path.join(data_directory, "excel_files_reduced_clean", f"{project_name}_reduced_clean.xlsx")
excel_raw_clean_filepath = os.path.join(data_directory, "excel_files_raw_clean", f"{project_name}_raw_clean.xlsx")
excel_reduced_clean_series_filepath = os.path.join(data_directory, "excel_files_reduced_clean_series", f"{project_name}_reduced_clean_series.xlsx")
excel_reduced_clean_series_python_filepath = os.path.join(data_directory, "excel_files_reduced_clean_series_python", f"{project_name}_reduced_clean_series_python.xlsx")
excel_raw_clean_series_python_filepath = os.path.join(data_directory, "excel_files_raw_clean_series_python", f"{project_name}_raw_clean_series_python.xlsx")

In [3]:
from formulas.tokens.operand import Range
from objects import Cell

from excel_utils import ExcelUtils
from typing import List


class SeriesImplementer:
        
    @staticmethod
    def implement_series(sheet_name, formula_ast_elements, series_mapping):
        """Implement series in formula AST"""
        for index, element in enumerate(formula_ast_elements):
            if isinstance(element, Range):
                cell_range_string = element.name

                _, cell_range = ExcelUtils.extract_cell_ranges_from_string(cell_range_string)
                
                cell_start_column, _, cell_end_column, _, _ = ExcelUtils.get_coordinates_from_range(cell_range)

                columns_in_range = list(range(cell_start_column, cell_end_column + 1))

                series_id_list = []
                
                for column in columns_in_range:
                    print(f"sheet_name: {sheet_name}, column: {column}")
        
                    series = series_mapping.get((sheet_name, column))
                    series_id_list.append(str(series.series_id))

                formula_ast_elements[index] = series_id_list
                
        return formula_ast_elements


In [4]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)
is_valid = ExcelValidator.validate_excel(excel_reduced)
if not is_valid:
    raise Exception("Excel file is not valid")

excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)
excel_raw_clean = ExcelCleaner.clean_excel(excel_raw)
ExcelBuilder.create_excel_from_workbook(excel_reduced_clean.workbook_with_formulas, excel_reduced_clean_filepath)
ExcelBuilder.create_excel_from_workbook(excel_raw_clean.workbook_with_formulas,  excel_raw_clean_filepath)

extracted_tables, workbook_data = TableExtractor.extract_tables(excel_reduced_clean)
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)
if not is_compatible:
    raise Exception("Excel file is not compatible")

series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, workbook_data=workbook_data)
series_mapping = SeriesMapper.map_series(series_dict)
series_iterator = SeriesIterator.iterate_series(series_dict)

series_list = [series for series in series_iterator]

series_list_with_formulas = [
    series for series in series_list if series.formulas != [None, None]
]
series_list_with_values = [
    series for series in series_list if series.formulas == [None, None]
]

generic_formula_dictionary = {}

for series in series_list_with_formulas:
    formula_1, formula_2 = series.formulas
    
    if formula_1 is not None and formula_2 is not None:

        formula_1_ast_elements, _ = FormulaParser.parse_formula(formula_1)
        formula_2_ast_elements, _ = FormulaParser.parse_formula(formula_2)

        generic_formula = FormulaGenerator.generate_generic_formula(formula_1_ast_elements, formula_2_ast_elements)

        sheet_name = series.series_id.sheet_name
        print(sheet_name)

        formula_1_ast_series = SeriesImplementer.implement_series(sheet_name, formula_1_ast_elements, series_mapping)

        print(formula_1_ast_series)

        generic_formula_dictionary[series.series_id] = (formula_1_ast_series, generic_formula)

summary
sheet_name: summary, column: 1
[['summary|Vehicle Reg |1|1']]
summary
sheet_name: summary, column: 1
sheet_name: summary, column: 3
sheet_name: summary, column: 3
[u- <Operator>, SUMIF <Function>, ( <Parenthesis>, ['summary|Vehicle Reg |1|1'], , <Operator>, ['summary|Vehicle Reg|1|3'], , <Operator>, ['summary|Vehicle Reg|1|3'], ) <Parenthesis>]
summary
sheet_name: summary, column: 1
[['summary|Vehicle Reg |1|1']]
summary
sheet_name: summary, column: 8
sheet_name: summary, column: 8
[['summary|TGB Cost (Inc VAT) |1|8'], - <Operator>, ( <Parenthesis>, ['summary|TGB Cost (Inc VAT) |1|8'], / <Operator>, 1.2 <Number>, ) <Parenthesis>]
summary
sheet_name: summary, column: 1
sheet_name: summary, column: 6
sheet_name: summary, column: 6
[u- <Operator>, SUMIF <Function>, ( <Parenthesis>, ['summary|Vehicle Reg |1|1'], , <Operator>, ['summary|Vehicle Reg|1|6'], , <Operator>, ['summary|Vehicle Reg|1|6'], ) <Parenthesis>]
xero
sheet_name: xero, column: 1
[CONCATENATE <Function>, ( <Parenthe

In [6]:
generic_formula_dictionary

{SeriesId(sheet_name='summary', series_header='Vehicle Reg', series_header_cell_row=1, series_header_cell_column=3): ([['summary|Vehicle Reg |1|1']],
  {0: (0, 1, 0, 1)}),
 SeriesId(sheet_name='summary', series_header='TGB Cost (No VAT) ', series_header_cell_row=1, series_header_cell_column=4): ([u- <Operator>,
   SUMIF <Function>,
   ( <Parenthesis>,
   ['summary|Vehicle Reg |1|1'],
   , <Operator>,
   ['summary|Vehicle Reg|1|3'],
   , <Operator>,
   ['summary|Vehicle Reg|1|3'],
   ) <Parenthesis>],
  {3: (0, 0, 0, 0), 5: (0, 1, 0, 1), 7: (0, 0, 0, 0)}),
 SeriesId(sheet_name='summary', series_header='Vehicle Reg', series_header_cell_row=1, series_header_cell_column=6): ([['summary|Vehicle Reg |1|1']],
  {0: (0, 1, 0, 1)}),
 SeriesId(sheet_name='summary', series_header='VAT', series_header_cell_row=1, series_header_cell_column=7): ([['summary|TGB Cost (Inc VAT) |1|8'],
   - <Operator>,
   ( <Parenthesis>,
   ['summary|TGB Cost (Inc VAT) |1|8'],
   / <Operator>,
   1.2 <Number>,
   ) <P